# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_file = "../cities_weather.csv"
weather_data_df = pd.read_csv(weather_data_file)



weather_data_df


,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hualmay,-11.0964,-77.6139,67.14,77,29,13.47,PE,1633635131
1,1,Rikitea,-23.1203,-134.9692,73.60,76,40,8.43,PF,1633635233
2,2,Ushuaia,-54.8000,-68.3000,49.66,43,0,5.75,AR,1633635234
3,3,Siguiri,11.6667,-9.5000,73.87,85,79,2.86,GN,1633635236
4,4,Albany,42.6001,-73.9662,76.32,71,0,1.54,US,1633635237
...,...,...,...,...,...,...,...,...,...,...
553,553,Sisimiut,66.9395,-53.6735,33.22,64,18,7.58,GL,1633636161
554,554,Gigant,46.5066,41.3419,44.71,54,27,18.03,RU,1633636168
555,555,Puerto Lleras,3.0223,-73.4044,83.35,77,87,1.66,CO,1633636169
556,556,Marawi,7.9986,124.2928,67.64,95,99,1.59,PH,1633636170


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data_df[["Lat", "Lon"]].astype(float)

weather_data_df = weather_data_df.dropna()
humidity = weather_data_df["Humidity"].astype(float)



In [5]:
fig = gmaps.figure()

heat_later = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=100,
                                  point_radius = 1)
fig.add_layer(heat_layer)
fig

ValueError: weights must be of the same length as locations or None

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_data_df = weather_data_df.loc[(weather_data_df["Max Temp"] >= 70) & 
                                       (weather_data_df["Max Temp"] <=80) &
                                       (weather_data_df["Wind Speed"] < 10) &
                                       (weather_data_df["Cloudiness"] == 0)]

weather_data_df
                                       
                               

,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Albany,42.6001,-73.9662,76.32,71,0,1.54,US,1633635237
11,11,Jalu,29.0331,21.5482,76.48,26,0,6.33,LY,1633635146
83,83,Tūkrah,32.5341,20.5791,74.21,60,0,5.28,LY,1633635370
161,161,Ubari,26.5921,12.7805,79.77,18,0,6.42,LY,1633635503
163,163,Karratha,-20.7377,116.8463,72.34,69,0,5.17,AU,1633635506
179,179,Upington,-28.4478,21.2561,73.78,17,0,8.05,ZA,1633635533
250,250,Miandrivazo,-19.5167,45.4667,74.37,55,0,4.81,MG,1633635652
472,472,Āsmār,35.0333,71.3581,70.00,37,0,5.86,AF,1633636027


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
weather_data_df['Hotel Name'] = ""
hotel_df = weather_data_df[["City Name", "Lat", "Lon", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date", "Hotel Name"]]
hotel_df

,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Albany,42.6001,-73.9662,76.32,71,0,1.54,US,1633635237,
11,Jalu,29.0331,21.5482,76.48,26,0,6.33,LY,1633635146,
83,Tūkrah,32.5341,20.5791,74.21,60,0,5.28,LY,1633635370,
161,Ubari,26.5921,12.7805,79.77,18,0,6.42,LY,1633635503,
163,Karratha,-20.7377,116.8463,72.34,69,0,5.17,AU,1633635506,
179,Upington,-28.4478,21.2561,73.78,17,0,8.05,ZA,1633635533,
250,Miandrivazo,-19.5167,45.4667,74.37,55,0,4.81,MG,1633635652,
472,Āsmār,35.0333,71.3581,70.00,37,0,5.86,AF,1633636027,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
